In [ ]:
import pandas as pd
import numpy as np
import importlib
import seaborn as sns
import matplotlib.pyplot as plt
import six
import sys
#print (sys.executable)
sys.modules['sklearn.externals.six'] = six
#!{sys.executable} -m pip install xgboost
#!{sys.executable} -m pip install imblearn
#!{sys.executable} -m pip install shap
import xgboost as xgb



#!pip3 install imblearn
#!pip3 install xgboost
#!pip3 install shap


from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
import shap

from filter_CRFS import Filter

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150

def clean_plot(leg=True, grid=None, font=None):
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    axis_color = 'lightgrey'
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['left'].set_color(axis_color)
    ax.tick_params(axis='both', color=axis_color)
    
    if leg:
        ax.legend(frameon = False, loc='upper left', bbox_to_anchor=(1, 1))
        
    if grid is not None:
        plt.grid(color='lightgrey', axis = grid, linestyle='-', linewidth=.5)
        
    if font is not None:
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
            ax.get_xticklabels() + ax.get_yticklabels()):
            
            item.set_fontfamily(font['family'])
            item.set_color(font['color'])
            

*REQUIRED FOR MACINE LEARNING MODEL - Generating Classifier from Apple/Cheese/Chicken/Corn_oil/Ginger/Mussel/Peanut/Potato/Soybean/Tomato/Wheat Data*

In [ ]:
atrain = pd.read_excel("Apple_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
atrain['food'] = 'apple'
chetrain = pd.read_excel("Cheese_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
chetrain['food'] = 'cheese'
chitrain = pd.read_excel("Chicken_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
chitrain['food'] = 'chicken'
cotrain = pd.read_excel("Corn_oil_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
cotrain['food'] = 'corn oil'
gtrain = pd.read_excel("Ginger_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
gtrain['food'] = 'ginger'
mtrain = pd.read_excel("Mussel_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
mtrain['food'] = 'mussel'
petrain = pd.read_excel("Peanut_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
petrain['food'] = 'peanut'
potrain = pd.read_excel("Potato_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
potrain['food'] = 'potato'
strain = pd.read_excel("Soybean_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
strain['food'] = 'soybean'
ttrain = pd.read_excel("Tomato_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
ttrain['food'] = 'tomato'
wtrain = pd.read_excel("Wheat_scoring.xlsx", engine='openpyxl')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
wtrain['food'] = 'wheat'


In [ ]:
atrain = atrain[atrain['abstract'].notnull()]
chetrain = chetrain[chetrain['abstract'].notnull()]
chitrain = chitrain[chitrain['abstract'].notnull()]
cotrain = cotrain[cotrain['abstract'].notnull()]
gtrain = gtrain[gtrain['abstract'].notnull()]
mtrain = mtrain[mtrain['abstract'].notnull()]
petrain = petrain[petrain['abstract'].notnull()]
potrain = potrain[potrain['abstract'].notnull()]
strain = strain[strain['abstract'].notnull()]
ttrain = ttrain[ttrain['abstract'].notnull()]
wtrain = wtrain[wtrain['abstract'].notnull()]


In [ ]:
amodel_data = Filter()
chemodel_data = Filter()
chimodel_data = Filter()
comodel_data = Filter()
gmodel_data = Filter()
mmodel_data = Filter()
pemodel_data = Filter()
pomodel_data = Filter()
smodel_data = Filter()
tmodel_data = Filter()
wmodel_data = Filter()



amodel_data.build_features(input_data = atrain, is_traindata = True)
chemodel_data.build_features(input_data = chetrain, is_traindata = True)
chimodel_data.build_features(input_data = chitrain, is_traindata = True)
comodel_data.build_features(input_data = cotrain, is_traindata = True)
gmodel_data.build_features(input_data = gtrain, is_traindata = True)
mmodel_data.build_features(input_data = mtrain, is_traindata = True)
pemodel_data.build_features(input_data = petrain, is_traindata = True)
pomodel_data.build_features(input_data = potrain, is_traindata = True)
smodel_data.build_features(input_data = strain, is_traindata = True)
tmodel_data.build_features(input_data = ttrain, is_traindata = True)
wmodel_data.build_features(input_data = wtrain, is_traindata = True)


In [ ]:
def balanced_sample(df, train_frac=.7, val_frac=.2):
    val_s1 = df[df['class'] == 0].sample(frac=val_frac)
    val_s2 = df[df['class'] == 1].sample(frac=val_frac)
    
    val= pd.concat([val_s1, val_s2], axis=0, ignore_index=True, sort=False)
    
    df = df.drop(val_s1.index.tolist() + val_s2.index.tolist(), axis = 0)
    
    y = df.copy()['class']
    x = df.copy().drop('class', axis = 1)
    
    #smote_x, smote_y = SMOTE().fit_sample(x, y)
    #sm = SMOTE(random_state=0)
    smote_x, smote_y = SMOTE().fit_resample(x, y)

    df = pd.DataFrame(smote_x, columns = x.columns)
    df['class'] = pd.Series(smote_y)
    
    train_s1 = df[df['class'] == 0].sample(frac=train_frac)
    train_s2 = df[df['class'] == 1].sample(frac=train_frac)
    
    test = df.drop(train_s1.index.tolist() + train_s2.index.tolist(), axis = 0)
    
    return pd.concat([train_s1, train_s2], axis=0, ignore_index=True, sort=False), test, val
    
adata = amodel_data.data.copy()
chedata = chemodel_data.data.copy()
chidata = chimodel_data.data.copy()
codata = comodel_data.data.copy()
gdata = gmodel_data.data.copy()
mdata = mmodel_data.data.copy()
pedata = pemodel_data.data.copy()
podata = pomodel_data.data.copy()
sdata = smodel_data.data.copy()
tdata = tmodel_data.data.copy()
wdata = wmodel_data.data.copy()

print(sum(adata['class'].tolist() + chedata['class'].tolist() + chidata['class'].tolist() + codata['class'].tolist() + gdata['class'].tolist()+ mdata['class'].tolist() + pedata['class'].tolist() + podata['class'].tolist() + sdata['class'].tolist() + tdata['class'].tolist() + wdata['class'].tolist()) / 
      len(adata['class'].tolist() + chedata['class'].tolist() + chidata['class'].tolist() + codata['class'].tolist() + gdata['class'].tolist()+ mdata['class'].tolist() + pedata['class'].tolist() + podata['class'].tolist() + sdata['class'].tolist() + tdata['class'].tolist() + wdata['class'].tolist()))
print(sum(adata['class'].tolist() + chedata['class'].tolist() + chidata['class'].tolist() + codata['class'].tolist() + gdata['class'].tolist()+ mdata['class'].tolist() + pedata['class'].tolist() + podata['class'].tolist() + sdata['class'].tolist() + tdata['class'].tolist() + wdata['class'].tolist())) 
print(len(adata['class'].tolist() + chedata['class'].tolist() + chidata['class'].tolist() + codata['class'].tolist() + gdata['class'].tolist()+ mdata['class'].tolist() + pedata['class'].tolist() + podata['class'].tolist() + sdata['class'].tolist() + tdata['class'].tolist() + wdata['class'].tolist()))


atrain, atest, aval = balanced_sample(adata)
chetrain, chetest, cheval = balanced_sample(chedata)
chitrain, chitest, chival = balanced_sample(chidata)
cotrain, cotest, coval = balanced_sample(codata)
gtrain, gtest, gval = balanced_sample(gdata)
mtrain, mtest, mval = balanced_sample(mdata)
petrain, petest, peval = balanced_sample(pedata)
potrain, potest, poval = balanced_sample(podata)
strain, stest, sval = balanced_sample(sdata)
ttrain, ttest, tval = balanced_sample(tdata)
wtrain, wtest, wval = balanced_sample(wdata)


train = pd.concat([atrain, chetrain, chitrain, cotrain, gtrain, mtrain, petrain, potrain, strain, ttrain, wtrain], axis=0, ignore_index=True, sort=False)
test = pd.concat([atest, chetest, chitest, cotest, gtest, mtest, petest, potest, stest, ttest, wtest], axis=0, ignore_index=True, sort=False)
val = pd.concat([aval, cheval, chival, coval, gval, mval, peval, poval, sval, tval, wval], axis=0, ignore_index=True, sort=False) # Does not incorporate SMOTE

In [ ]:
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as MSE

y = train.copy()['class']
x = train.copy().drop('class', axis = 1)

model = XGBClassifier(learning_rate =0.1,
 n_estimators=300,
 #max_depth=5,
 max_depth=2,
 #min_child_weight=1,
 min_child_weight=2,
 #gamma=0,
 gamma=0.2,
 #subsample=0.8,
 subsample=0.9,
 #colsample_bytree=0.8,
 colsample_bytree=0.95,
 reg_alpha=1e-05,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

model.fit(x, y)

scores = cross_val_score(model, x, y, cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())
 
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(model, x, y, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean()) 

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# RMSE Computation
rmse = np.sqrt(MSE(y_test, predictions))
print("RMSE : % f" %(rmse))

# Adjusted R-squared
adjsR = 1 - (1-model.score(x, y))*(len(y)-1)/(len(y)-x.shape[1]-1)
print("adj-R2 : % f" %(adjsR))


# Confusion matrix
cm1 = metrics.confusion_matrix(y_test, predictions)
print(cm1)
total1=sum(sum(cm1))

#####from confusion matrix calculate accuracy
accuracy1=(cm1[0,0]+cm1[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)

In [ ]:
pd.DataFrame({'feature' : x.columns, 'importance' : model.feature_importances_}).sort_values('importance', ascending=False)

*Feature Extraction from Papers - CHANGE FILES AFTER THIS*

In [ ]:
# Change according to file/food
Xtrain = pd.read_csv('C:\\...Data\\.....csv', encoding='latin1')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms']]
Xtrain['food'] = 'pasta'

In [ ]:
Xtrain['is_useful']=1
Xtrain = Xtrain[Xtrain['is_useful'].notnull()]
Xtrain = Xtrain[Xtrain['abstract'].notnull()]
Xtrain['is_useful'] = Xtrain['is_useful'].replace(2, 1, regex=True)
Xtrain['is_useful'] = Xtrain['is_useful'].replace(0, 1, regex=True)

#training_data = pd.concat([gtrain, ctrain], axis=0, ignore_index = True)

In [ ]:
Xmodel_data = Filter()

Xmodel_data.build_features(input_data = Xtrain, is_traindata = True)

*Classifier application to Papers*

In [ ]:
X_df = Xmodel_data.data

X_df_class_dropped = X_df.drop(columns=['class'])
#X_df_class_dropped

In [ ]:
predicted_probs = model.predict(X_df_class_dropped)
#print(list(predicted_probs))

list(predicted_probs).count(1.0)

In [ ]:
X_df_class_dropped['Estimated class'] = pd.Series(predicted_probs, index=X_df_class_dropped.index)

In [ ]:
# Change according to file/food
final_pd = pd.read_csv("C:\\...\\Data\\....csv", encoding='latin1')[['PMID', 'abstract', 'paper', 'journal', 'webpage','year']]

In [ ]:
final_pd = final_pd[final_pd['abstract'].notnull()]

final_pd['Estimated class'] = pd.Series(predicted_probs, index=final_pd.index)

In [ ]:
from pandas import ExcelWriter

# Change according to file/food
writer = ExcelWriter('C:\\...\\Ranking\\..._Papers_Estimated_Class.xlsx')#
final_pd.to_excel(writer,'Pasta_Papers')#
writer.save()